In [1]:
import sys
import os
import numpy as np
import math
import imp

In [2]:
sys.path.insert(0, os.path.expanduser('~/git/refl1d_analysis/notebooks'))
import plot_utils

In [13]:
import refnx
import refl1d
from refl1d.names import *

from refnx.reflect._creflect import abeles as refnx_abeles

from refl1d.abeles import refl
from refl1d.reflectivity import reflectivity

## The behavior of sqrt(-A + iB) where A>0 and B=0 is different in numpy and C.

Numpy produces a positive value for the imaginary part, and C chooses the negative solution.

In [4]:
print("sqrt(-3 + i*0) = %s" % np.sqrt(-3.+1j*0))

print("sqrt(-3 + i*1e-30) = %s" % np.sqrt(-3.+1j*1e-30))

print("sqrt(-3 - i*1e-30) = %s" % np.sqrt(-3.-1j*1e-30))

sqrt(-3 + i*0) = 1.73205080757j
sqrt(-3 + i*1e-30) = (2.88675134595e-31+1.73205080757j)
sqrt(-3 - i*1e-30) = (2.88675134595e-31-1.73205080757j)


## Adding a tiny absorption to the backing mediumis enough to force the choice

The problem appears we have three media with increasing SLD.


In [35]:
def magsq(z):
    return z.real**2 + z.imag**2


q = np.linspace(0.008, 0.05, 500)
epsilon = 0.000136 # Representative of actual D20 absorption

depth =        [0,      850, 0]
rho =          [2.067,  4.3, 6.]
irho_zero =    [0.,     0.1, 0.]
irho =         [0.0,    0.1, epsilon]
refl1d_sigma =         [35,  5.]
refnx_sigma =  [np.nan, 35,  5.]

w = np.c_[depth, rho, irho, refnx_sigma]
w_zero = np.c_[depth, rho, irho_zero, refnx_sigma]

r_nx = refnx_abeles(q, w)
r_nx_zero = refnx_abeles(q, w_zero)

r_1d_abeles = magsq(refl(q/2.0, depth, rho, irho=irho, sigma=refl1d_sigma))
r_1d_abeles_zero = magsq(refl(q/2.0, depth, rho, irho=irho_zero, sigma=refl1d_sigma))

r_1d = reflectivity(q/2.0, depth, rho, irho=irho, sigma=refl1d_sigma)


In [36]:
plot_utils.plot1d([[q, r_nx], [q, r_nx_zero], [q, r_1d_abeles], [q, r_1d_abeles_zero], [q, r_1d]],
                  data_names=['refnx Im=epsilon', 'refnx Im=0', 'abeles Im=epsilon', 'abeles Im=0', 'refl1d'],
                  y_log=True, x_log=True)